In [4]:
from numpy import array
from numpy import hstack
from keras.preprocessing.sequence import TimeseriesGenerator


In [6]:

in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95, 105])

In [7]:

in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))



In [8]:
dataset = hstack((in_seq1, in_seq2))
print(dataset)


[[ 10  15]
 [ 20  25]
 [ 30  35]
 [ 40  45]
 [ 50  55]
 [ 60  65]
 [ 70  75]
 [ 80  85]
 [ 90  95]
 [100 105]]


In [39]:
n_input = 3
generator = TimeseriesGenerator(dataset, dataset, length=n_input, batch_size=1)
print('Samples: %d' % len(generator))

Samples: 6


In [40]:
for i in range(len(generator)):
    x, y = generator[i]
    print('%s => %s' % (x, y))

[[[10 15 25]
  [20 25 45]
  [30 35 65]]] => [[40 45 85]]
[[[20 25 45]
  [30 35 65]
  [40 45 85]]] => [[ 50  55 105]]
[[[ 30  35  65]
  [ 40  45  85]
  [ 50  55 105]]] => [[ 60  65 125]]
[[[ 40  45  85]
  [ 50  55 105]
  [ 60  65 125]]] => [[ 70  75 145]]
[[[ 50  55 105]
  [ 60  65 125]
  [ 70  75 145]]] => [[ 80  85 165]]
[[[ 60  65 125]
  [ 70  75 145]
  [ 80  85 165]]] => [[ 90  95 185]]


# OR

In [19]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [21]:
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        end_ix = i + n_steps
        if end_ix > len(sequences):
            break
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [22]:
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])


In [23]:
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

In [24]:
dataset = hstack((in_seq1, in_seq2, out_seq)) 

In [32]:
n_steps=3


In [33]:
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)


(7, 3, 2) (7,)


In [34]:
for i in range(len(X)):
    print(X[i], y[i])

[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


In [35]:
X, y = split_sequences(dataset, n_steps)
n_features = X.shape[2]

In [36]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [37]:
model.fit(X, y, epochs=200, verbose=0)

In [38]:
x_input = array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[206.06271]]
